In [1]:
# @title Instalação do deap
! pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.2 MB/s eta 0:00:00


In [38]:
# @title Download dos dados de entrada

!wget http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/atsp/ft53.atsp.gz
!gunzip ft53.atsp.gz

!wget http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/atsp/rbg443.atsp.gz
!gunzip rbg443.atsp.gz

--2025-03-06 14:41:24--  http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/atsp/ft53.atsp.gz
Resolving comopt.ifi.uni-heidelberg.de (comopt.ifi.uni-heidelberg.de)... 129.206.106.221
Connecting to comopt.ifi.uni-heidelberg.de (comopt.ifi.uni-heidelberg.de)|129.206.106.221|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6031 (5.9K) [application/octet-stream]
Saving to: ‘ft53.atsp.gz’

ft53.atsp.gz        100%[===================>]   5.89K  --.-KB/s    in 0s      

2025-03-06 14:41:28 (79.6 MB/s) - ‘ft53.atsp.gz’ saved [6031/6031]

gzip: ft53.atsp already exists; do you wish to overwrite (y or n)? ^C
--2025-03-06 14:41:35--  http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/atsp/rbg443.atsp.gz
Resolving comopt.ifi.uni-heidelberg.de (comopt.ifi.uni-heidelberg.de)... 129.206.106.221
Connecting to comopt.ifi.uni-heidelberg.de (comopt.ifi.uni-heidelberg.de)|129.206.106.221|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103674 (101K) [

In [49]:
# @title Import de Bibliotecas

import numpy as np
import matplotlib.pyplot as plt
import random
from deap import base
from deap import creator
from deap import tools
from deap import algorithms

In [43]:
# @title Função p/ leitura dos dados de entrada

# Retorna uma tupla (distancias, ncidades)
def read_data(fpath):
  distances, n_cities = None, None

  with open(fpath, "r") as f:
      lines = f.readlines()
      starting_i = 0
      for line in lines:
        starting_i +=1
        if 'EDGE_WEIGHT_SECTION' in line:
          break

        if 'DIMENSION' in line:
          n_cities = int(line.split()[-1])

      distances = np.zeros((n_cities,n_cities), dtype=np.int64)
      i, j = 0, 0
      for line in lines[starting_i:]:
        if 'EOF' in line or i >= n_cities:
          break

        for distance in line.split():
          if i==j: # Tratando os dados para facilitar a implementação
            distances[i][j] = 9999999 # Sair de i e ir para i é indesejável
          else:
            distances[i][j] = int(distance)
          j += 1
          if j >= n_cities:
            j = 0
            i += 1
  return distances, n_cities

In [45]:
filepath = 'ft53.atsp' # @param {'type':'string'}
distances, n_cities = read_data(filepath)

print(f"Arquivo {filepath} lido com sucesso!")
print(f"Número de cidades: {n_cities}")
print("Matriz de distâncias:")
print(distances)


Arquivo ft53.atsp lido com sucesso!
Número de cidades: 53
Matriz de distâncias:
[[9999999     223     210 ...     357     420     344]
 [     58 9999999     179 ...     300     214     314]
 [     60     109 9999999 ...     201     302     190]
 ...
 [   1504    1464    1629 ... 9999999     272     159]
 [   1505    1534    1545 ...     106 9999999     240]
 [   1520    1472    1612 ...      29     118 9999999]]


In [46]:
# @title Minimização da FO

# Problema de minimização
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

# Indivíduo derivado de uma permutação
creator.create("Individual", list, fitness=creator.FitnessMin)

/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [47]:
# @title Criação da Toolbox + registros do indivíduo e população
toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(n_cities), n_cities) # A solução é representada como um permutação
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual) # População é uma lista de indivíduos

In [48]:
# @title Função Objetivo

def fitness(individual):
  distance = 0 # Distância total viajada
  for i in range(n_cities-1):
    distance += distances[individual[i]][individual[i+1]]
  distance += distances[individual[n_cities-1]][individual[0]] # Volta para a cidade original

  return distance, # Como a solução é uma permutação, não há necessidade de preocupar-se com infrações das restrições

In [53]:
# @title Registro da FO + operadores (cruzamento, mutação e seleção)

toolbox.register("evaluate", fitness)
toolbox.register("mate", tools.cxOrdered)  # Crossover ordenado
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)  # Mutação por troca de cidades
toolbox.register("select", tools.selTournament, tournsize=3)

In [54]:
def main():
  # SEED = 89
  # random.seed(SEED)
  pop = toolbox.population(n=100)  # População inicial
  hof = tools.HallOfFame(1)  # Melhor solução encontrada
  stats = tools.Statistics(lambda ind: ind.fitness.values)
  stats.register("min", np.min)
  stats.register("avg", np.mean)

  # Evolução com algoritmo genético
  algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.2, ngen=200, stats=stats, halloffame=hof, verbose=True)

  print("\nMelhor rota encontrada:", hof[0])
  print("Distância total:", fitness(hof[0])[0])

In [55]:
main()

gen	nevals	min  	avg    
0  	100   	22732	26086.2
1  	78    	22358	25434.8
2  	69    	22761	25027.8
3  	79    	22680	24897.2
4  	80    	22206	24717.7
5  	82    	21910	24517.7
6  	73    	21467	24017.8
7  	79    	20779	24031.8
8  	69    	20779	23618.7
9  	79    	20292	23146.3
10 	80    	20292	23042.8
11 	77    	20292	22643.4
12 	82    	19717	22422.7
13 	76    	18531	22145.7
14 	71    	18918	22037.6
15 	67    	18918	21821.1
16 	75    	19470	21522.5
17 	79    	19470	21389.3
18 	74    	19364	21547.8
19 	81    	18506	21416.9
20 	77    	18506	21224.9
21 	83    	18142	21096.5
22 	74    	18424	20784.6
23 	77    	18424	21134  
24 	73    	18218	20928.8
25 	84    	17634	21115.7
26 	76    	17634	20801.2
27 	70    	17634	20426.4
28 	80    	18084	20481.1
29 	84    	18294	20240.4
30 	89    	17657	20010.6
31 	74    	17070	19984.4
32 	69    	17070	19748  
33 	84    	17070	19695.2
34 	83    	17070	19377.5
35 	80    	16879	19415.8
36 	76    	16881	19001.6
37 	80    	16889	19115  
38 	70    	16881	19148.8
